In [ ]:
import coloredlogs, logging
import torch
import sys
sys.path.append("..")
logger = logging.getLogger(__name__)
coloredlogs.install(level='DEBUG')

In [ ]:
aa = torch.Tensor([1,2,3,1,2,3,4,5,6,7,8,1,3,4,5,6,7,8,1])

In [ ]:
from collections import defaultdict
a = defaultdict(lambda: [])
a['a']=123
a['b']=124
a['c']=125

In [ ]:
print(a.values())

In [ ]:
from federated_learning.helper import utils
bb = utils.attack_shuffle_labels(aa,1)

In [ ]:
print(aa)
print(bb)

In [ ]:
# import neptune
# token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNmRhYmZmM2YtZDc5Yi00ZGMyLWE5MWEtMjVkMDEwOGNjYTliIn0="
# neptune.init(project_qualified_name="ehsan/sandbox", api_token=token)
# neptune.create_experiment(name="Results Evaluation")


In [ ]:
neptune.append_tag("EHSAN ETESAMI")

In [ ]:
neptune.stop()

In [1]:
import sys
sys.path.append("..")
from federated_learning.helper import utils
from federated_learning.FLNet import FLNet
from federated_learning.Arguments import Arguments
import torch, random
from collections import defaultdict
# import memory_profiler
import copy
from torchvision import transforms
from federated_learning.FLCustomDataset import FLCustomDataset
import coloredlogs, logging
CONFIG_PATH = '../configs/defaults.yml'


In [2]:
dd = utils.load_mnist_dataset(
                        train=True, 
                        transform=transforms.Compose([transforms.ToTensor(),]))

In [6]:
print(dd.data[0])

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,
          18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   

In [ ]:
configs = utils.load_config(CONFIG_PATH)
args = Arguments(
        shards_num=configs['mnist']['shards_num'],
        shards_per_worker_num=configs['mnist']['shards_per_worker_num'],
        total_users_num=configs['mnist']['total_users_num'],
        log_level="DEBUG",
        log_format='%(asctime)s: %(message)s'
    )
workers_idx = ["worker_" + str(i) for i in range(args.total_users_num)]
logger = logging.getLogger(__name__)
coloredlogs.install(level=args.log_level, fmt=args.log_format)

In [ ]:
mapped_datasets = dict()

mapped_ds_itr = utils.map_shards_to_worker(
    utils.split_randomly_dataset(
        utils.load_mnist_dataset(
            train=True, 
            transform=transforms.Compose([transforms.ToTensor(),])),
        args.shards_num),
    workers_idx, 
    args.shards_per_worker_num)

for mapped_ds in mapped_ds_itr:
    for ww_id, dataset in mapped_ds.items():
        mapped_datasets.update(mapped_ds)

In [ ]:
mapped_datasets

In [ ]:
mapped_datasets = dict()

mapped_ds_itr = utils.map_shards_to_worker(
    utils.split_randomly_dataset(
        utils.load_mnist_dataset(
            train=True, 
            transform=transforms.Compose([transforms.ToTensor(),])),
        args.shards_num),
    workers_idx, 
    args.shards_per_worker_num)

for mapped_ds in mapped_ds_itr:
    mapped_datasets.update(mapped_ds)

In [ ]:
mapped_datasets

In [2]:
import torch
import numpy as np

In [3]:
aa = torch.rand((5, 3, 3))

In [4]:
cc = np.random.uniform(0, 4, (5,3,3))

In [5]:
bb = [0,2,4]
gg = aa[bb]
kk = (aa[bb] + cc[bb]).byte()

In [6]:
print(gg.type())
print(kk.type())

torch.FloatTensor
torch.ByteTensor


In [7]:
gg

tensor([[[0.5257, 0.1200, 0.5131],
         [0.4211, 0.4110, 0.3703],
         [0.4994, 0.1466, 0.0668]],

        [[0.6981, 0.7449, 0.1485],
         [0.3797, 0.8342, 0.5889],
         [0.2707, 0.8610, 0.8591]],

        [[0.2741, 0.5852, 0.1194],
         [0.8342, 0.6104, 0.5135],
         [0.2620, 0.0622, 0.5765]]])

In [8]:
kk

tensor([[[3, 2, 3],
         [3, 2, 4],
         [2, 2, 3]],

        [[2, 0, 0],
         [2, 1, 4],
         [1, 2, 1]],

        [[3, 3, 3],
         [2, 0, 1],
         [3, 4, 3]]], dtype=torch.uint8)

In [ ]:
aa = torch.tensor([1,2,3,4,5,6,7,8,9])
bb = [1,4,2,7]

print(aa[bb])
print()
print(aa[bb] + np.random.uniform(0, 4, (4,5,5)))

In [ ]:
aa

In [ ]:
def test1(parameter, **args):
    print(parameter)
    print(args.items())
    for ii, jj in args.items():
        print(ii, jj)

In [ ]:
def test2(parameter, *args):
    print(parameter)
    print(args)

In [ ]:
test1("ehsan", **args)

In [ ]:
test2("ehsan", ['ehsan','etesami', 1234])

In [ ]:
import matplotlib.pyplot as plt
def draw_samples(data):
    list_keys = list(data.keys())
    data_np = dict()
    data_np['x'] = np.array(data[list_keys[0]]['x'], dtype = np.float32).reshape(-1, 28, 28)
    data_np['y'] = np.array(data[list_keys[0]]['y'], dtype = np.int64).reshape(-1, 1)
    figure = plt.figure(figsize=(8,8))
    for i in range(36):
        plt.subplot(6, 6, i + 1)
        plt.axis("off")
        plt.title(data_np['y'][i])
        plt.imshow(data_np['x'][i], cmap='gray')
        
draw_samples(raw_test_data)